In [173]:
import pandas as pd
import re
pd.options.display.float_format = '{:.0f}'.format

In [174]:
#Conversión de CSVs de contratos ComprasMX a dfs.
contratos = []

for i in range(2012,2026):
    contrato = pd.read_csv('Contratos_CompraNet_{}.csv'.format(i), encoding='ISO-8859-1',low_memory=False, on_bad_lines='skip')
    contratos.append(contrato)

In [175]:
#Loop para modificación de columnas de dfs (2010-2017)
for i in range(0,6):
     contratos[i] = contratos[i].rename(columns={'SIGLAS':'Siglas de la Institución',
'CLAVEUC': 'Clave de la UC',
'NOMBRE_DE_LA_UC':'Nombre de la UC',
'TIPO_PROCEDIMIENTO':'Tipo Procedimiento',
'CODIGO_CONTRATO':'Código del contrato',
'TITULO_CONTRATO':'Título del contrato',
'IMPORTE_CONTRATO':'Importe DRC',
'MONEDA':'Moneda',
'PROVEEDOR_CONTRATISTA':'Proveedor o contratista',
'SIGLAS_PAIS':'País de la empresa',
'ESTRATIFICACION_MPC':'Estratificación',
'ANUNCIO':'Dirección del anuncio',}
                                       )

In [176]:
#Loop para modificación de columnas de dfs (2018-2022)
for i in range(6,12):
     contratos[i] = contratos[i].rename(columns={'Importe del contrato':'Importe DRC',
'Moneda del contrato':'Moneda',
'Clave del país de la empresa':'País de la empresa',
'Estratificación de la empresa':'Estratificación',
'Tipo de procedimiento': 'Tipo Procedimiento'})

In [178]:
#Loop para modificación de columnas de dfs en contratos
for i in range(0,len(contratos)):
    contratos[i] = contratos[i][['Siglas de la Institución', 'Clave de la UC', 'Nombre de la UC',
'Tipo Procedimiento', 'Código del contrato', 
'Título del contrato', 'Importe DRC', 
'Moneda', 'Proveedor o contratista', 'País de la empresa', 
'Estratificación', 'Dirección del anuncio']]

In [179]:
#Añadir columna "Años" a dfs en lista contratos
años = list(range(2012, 2026))

for df, año in zip(contratos, años):
    df['Año'] = año

C:\Users\kazrc\AppData\Local\Temp\ipykernel_7680\761244251.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Año'] = año


In [180]:
#Stack de todos los dfs en contratos
contratos_stack = pd.concat(contratos, axis=0)

In [181]:
#Lista de proveedores pertenecientes a GRUPO ANDRADE
grupo_andrade = ['ANGAR AZCAPOTZALCO SA DE CV', 'Angar Azcapotzalco SA de CV',
'ANGAR AZCAPOTZALCO', 'Angar Azcapotzalco',
'DISTRIBUIDORA DE VEHICULOS GA DE MX SA de CV', 'Distribuidora de Vehiculos GA de Mx SA de CV',
'AUTOANGAR SA DE CV', 'Autoangar SA de CV',
'AUTOMOVILISTICA ANDRADE SA DE CV', 'Automovilistica Andrade SA de CV',
'Andrade Universidad SA de CV', 'ANDRADE UNIVERSIDAD SA DE CV',
'Distribuidora de Vehiculos Coreanos Andrade SA de CV', 'DISTRIBUIDORA DE VEHICULOS COREANOS ANDRADE SA DE CV',
'Distribuidora Lomas Anahuac SA de CV','DISTRIBUIDORA LOMAS ANAHUAC SA DE CV',       
'Zaragoza Motriz SA de CV','ZARAGOZA MOTRIZ SA DE CV',
'Camiones Repuestos y Accesorios SA de CV', 'CAMIONES REPUESTOS Y ACCESORIOS SA DE CV',
'Camiones, Repuestos y Accesorios SA DE CV','CAMIONES, REPUESTOS Y ACCESORIOS SA DE CV',
'Andrade Zaragoza SA de CV', 'ANDRADE ZARAGOZA SA DE CV',
'Andrade Comercializadora Vehicular SA de CV', 'ANDRADE COMERCIALIZADORA VEHICULAR SA DE CV',
'Andrade Distribuidora Automotriz Chinos SA de CV', 'ANDRADE DISTRIBUIDORA AUTOMOTRIZ CHINOS SA DE CV',
'Distribuidora de Vehículos GA de México S.A. de C.V', 'DISTRIBUIDORA DE VEHÍCULOS GA DE MÉXICO S.A. DE C.V.',
'Comercializadora Vehicular Huixquilucan, S.A. de C.V.', 'COMERCIALIZADORA VEHICULAR HUIXQUILUCAN, S.A. DE C.V.',
'INTEGRA ARRENDA SA DE CV SOFOM ENR']

#Función para incorporar empresas de Grupo Andrade bajo mismo nombre
def filtro_andrade(df, grupo_andrade, nuevo):
    df = df.copy()
    df['Proveedor o contratista'] = df['Proveedor o contratista'].replace(grupo_andrade, nuevo)
    return df

In [182]:
contratos_stack = filtro_andrade(contratos_stack, grupo_andrade, 'GRUPO ANDRADE')

In [184]:
contratos_stack['Importe DRC'] = (
    contratos_stack['Importe DRC']
    .astype(str)
    .str.replace(',', '', regex=False)
    .str.replace('$', '', regex=False)
)

contratos_stack['Importe DRC'] = pd.to_numeric(
    contratos_stack['Importe DRC'], errors='coerce'
)

In [185]:
contratos_stack_andrade = contratos_stack.loc[contratos_stack['Proveedor o contratista'] == 'GRUPO ANDRADE']
contratos_stack_andrade.head()

,Siglas de la Institución,Clave de la UC,Nombre de la UC,Tipo Procedimiento,Código del contrato,Título del contrato,Importe DRC,Moneda,Proveedor o contratista,País de la empresa,Estratificación,Dirección del anuncio,Año
8284,CONANP,016F00001,CONANP-Dirección Ejecutiva de Administración y...,Licitación Pública,65760,ADQUISICION DE CAMIONETAS Y EMBARCACIONES,7310000,MXN,GRUPO ANDRADE,MX,No MIPYME,https://historico-compranet.buengobierno.gob.m...,2012
17517,SEMARNAT,16000997,SEMARNAT-Dirección de Adquisiciones y Contrato...,Invitación a Cuando Menos 3 Personas,126567,ADQUISICIÓN DE VEHÍCULOS TERRESTRES,13342900,MXN,GRUPO ANDRADE,MX,No MIPYME,https://historico-compranet.buengobierno.gob.m...,2012
20317,PROFEPA,1.60E+02,PROFEPA-Dirección de Recursos Materiales y Ser...,Licitación Pública,204036,ADQUISICIÓN DE VEHÍCULOS NUEVOS,3399000,MXN,GRUPO ANDRADE,MX,No MIPYME,https://historico-compranet.buengobierno.gob.m...,2012
37643,PUE,921002997,PUE-Secretaría De Finanzas y Administración-Di...,Licitación Pública,302326,PUEBLA ADQUISICIÓN VEHÍCULOS SUBDIR EPIDEM SER...,921388,MXN,GRUPO ANDRADE,MX,No MIPYME,https://historico-compranet.buengobierno.gob.m...,2012
77190,CDMX,909024988,CDMX-Procuraduría Ambiental y del Ordenamiento...,Licitación Pública,231542,ADQUISICIÓN DE CAMIONETAS,2193103,MXN,GRUPO ANDRADE,MX,No MIPYME,https://historico-compranet.buengobierno.gob.m...,2012


In [186]:
#GroupBy de todos los contratos con Grupo Andrade por institución y monto de contrato.
andrade_compradores_principales = contratos_stack_andrade.groupby(['Proveedor o contratista', 'Siglas de la Institución']).agg({'Importe DRC':'sum'}).sort_values(by='Importe DRC', ascending=False)                       
andrade_compradores_principales.columns = ['Importe DRC']
andrade_compradores_principales = andrade_compradores_principales.reset_index()
andrade_compradores_principales.head()                                                         

,Proveedor o contratista,Siglas de la Institución,Importe DRC
0,GRUPO ANDRADE,SEDENA,12124409535
1,GRUPO ANDRADE,BIENESTAR,7283155340
2,GRUPO ANDRADE,IMSS,4241869510
3,GRUPO ANDRADE,CAPUFE,1270088189
4,GRUPO ANDRADE,ISSSTE,1081422888


In [187]:
#GroupBy de todos los contratos con Grupo Andrade por institución y número de contratos
andrade_compradores_conteo = contratos_stack_andrade.groupby(['Proveedor o contratista', 'Siglas de la Institución']).agg({'Siglas de la Institución':'count'})
andrade_compradores_conteo.columns = ['Conteo Contratos']
andrade_compradores_conteo = andrade_compradores_conteo.reset_index()
andrade_compradores_conteo = andrade_compradores_conteo.sort_values(by ='Conteo Contratos', ascending=False)
andrade_compradores_conteo.head()

,Proveedor o contratista,Siglas de la Institución,Conteo Contratos
67,GRUPO ANDRADE,SEDENA,38
20,GRUPO ANDRADE,CONAGUA,8
14,GRUPO ANDRADE,CFE,8
10,GRUPO ANDRADE,CAPUFE,7
70,GRUPO ANDRADE,SEMAR,7


In [202]:
#Módulos necesarios para diagrama Sankey
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'

In [225]:
#nodos, sources, targets y colores para Saneky
dependencias = andrade_compradores_principales['Siglas de la Institución'].head(10).tolist()
empresa = 'Grupo Andrade'
nodos = dependencias + [empresa]

source_indices = list(range(len(dependencias))) 
target_indices = [len(dependencias)] * len(dependencias)

valores = andrade_compradores_principales['Importe DRC'].tolist()

flow_colors = ['rgba(255, 0, 0, 0.9)',
               'rgba(255, 0, 0, 0.6)',
               'rgba(255, 0, 0, 0.3)',
               'rgba(128, 128, 128, 0.6)',
               'rgba(128, 128, 128, 0.6)',
               'rgba(128, 128, 128, 0.6)',
               'rgba(128, 128, 128, 0.6)',
               'rgba(128, 128, 128, 0.6)',
               'rgba(128, 128, 128, 0.6)',
               'rgba(128, 128, 128, 0.6)']

In [226]:
#Diagrama Sankey
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=nodos,
        pad=15,
        thickness=20,
        color="lightblue"
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=valores,
        color= flow_colors
    )
)])

fig.update_layout(title_text=f"Grupo Andrade: Clientes predilectos", font_size=12)
fig.show()